## Data preprocessing
此份檔案包含將資料整理為50萬名使用者在各月份的交易金額、次數、以及其他相關特徵(long to wide format)，供後續跑sliding window使用。
另外在計算最常使用的卡號、國內外/線上線下消費紀錄時會分別取原始資料相對應的cnt / amt_pct 做特徵，因此目前有三個版本的整理後資料(在檔名上略有不同)。

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pred_class = ['2', '6', '10', '12', '13', '15', '18', '19', '21', '22', '25', '26', '36', '37', '39', '48']

In [ ]:
# 使用cnt作為統計 card 與 國內外/線上線下消費的特徵

use_cols = ['dt', 'chid', 'shop_tag', 'txn_cnt', 'txn_amt', 'domestic_offline_cnt', 'domestic_online_cnt',
            'overseas_offline_cnt', 'overseas_online_cnt','card_1_txn_cnt', 'card_2_txn_cnt', 'card_3_txn_cnt',
            'card_4_txn_cnt', 'card_5_txn_cnt', 'card_6_txn_cnt', 'card_7_txn_cnt', 'card_8_txn_cnt',
            'card_9_txn_cnt', 'card_10_txn_cnt', 'card_11_txn_cnt', 'card_12_txn_cnt', 'card_13_txn_cnt',
            'card_14_txn_cnt', 'card_other_txn_cnt']

df = pd.read_csv('tbrain_cc_training_48tags_hash_final.csv', usecols=use_cols)

df = df[df.shop_tag.isin(pred_class)]
df['max_card_idx'] = np.argmax(df.values[:,9:], axis = 1)+1

all_chid_list = [i for i in range(10000000, 10000000+500000)]

data = pd.DataFrame(np.zeros((500000, 24 * len(pred_class) * 7)))
data.columns = [f"txn_{k}_dt{i}_shoptag_{j}" for i in range(1, 25) for j in pred_class for k in ['amt', 'cnt', 'card', 'off_do', 'on_do', 'off_ov', 'on_ov']]
data["chid"] = all_chid_list

data.index = data.chid
data.drop("chid", inplace=True, axis=1)

for (date, shop_tag), date_shoptag_data in df.groupby(["dt", "shop_tag"]):
    data.loc[date_shoptag_data.chid.values, [f"txn_amt_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.txn_amt.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_cnt_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.txn_cnt.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_card_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.max_card_idx.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_off_do_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.domestic_offline_cnt.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_on_do_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.domestic_online_cnt.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_off_ov_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.overseas_offline_cnt.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_on_ov_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.overseas_online_cnt.values.reshape(-1,1)

data.to_csv('cnt_amt_card_cnt_onoff_doov_cnt_data.csv')

In [ ]:
# 使用cnt作為國內外/線上線下消費的特徵，card則以amt_pct做計算取最大使用量的卡號

use_cols = ['dt', 'chid', 'shop_tag', 'txn_cnt', 'txn_amt', 'domestic_offline_cnt', 'domestic_online_cnt',
            'overseas_offline_cnt', 'overseas_online_cnt','card_1_txn_amt_pct', 'card_2_txn_amt_pct', 'card_3_txn_amt_pct',
            'card_4_txn_amt_pct', 'card_5_txn_amt_pct', 'card_6_txn_amt_pct', 'card_7_txn_amt_pct', 'card_8_txn_amt_pct',
            'card_9_txn_amt_pct', 'card_10_txn_amt_pct', 'card_11_txn_amt_pct', 'card_12_txn_amt_pct', 'card_13_txn_amt_pct',
            'card_14_txn_amt_pct', 'card_other_txn_amt_pct']

df = pd.read_csv('tbrain_cc_training_48tags_hash_final.csv', usecols=use_cols)

df = df[df.shop_tag.isin(pred_class)]
df['max_card_idx'] = np.argmax(df.values[:,9:], axis = 1)+1

all_chid_list = [i for i in range(10000000, 10000000+500000)]

data = pd.DataFrame(np.zeros((500000, 24 * len(pred_class) * 7)))
data.columns = [f"txn_{k}_dt{i}_shoptag_{j}" for i in range(1, 25) for j in pred_class for k in ['amt', 'cnt', 'card', 'off_do', 'on_do', 'off_ov', 'on_ov']]
data["chid"] = all_chid_list

data.index = data.chid
data.drop("chid", inplace=True, axis=1)

for (date, shop_tag), date_shoptag_data in df.groupby(["dt", "shop_tag"]):
    data.loc[date_shoptag_data.chid.values, [f"txn_amt_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.txn_amt.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_cnt_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.txn_cnt.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_card_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.max_card_idx.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_off_do_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.domestic_offline_cnt.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_on_do_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.domestic_online_cnt.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_off_ov_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.overseas_offline_cnt.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_on_ov_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.overseas_online_cnt.values.reshape(-1,1)

data.to_csv('cnt_amt_card_onoff_doov_cnt_data.csv')

In [ ]:
# 使用amt_pct作為統計 card 與 國內外/線上線下 消費的特徵

use_cols = ['dt', 'chid', 'shop_tag', 'txn_cnt', 'txn_amt', 'domestic_offline_amt_pct', 'domestic_online_amt_pct',
            'overseas_offline_amt_pct', 'overseas_online_amt_pct','card_1_txn_amt_pct', 'card_2_txn_amt_pct', 'card_3_txn_amt_pct',
            'card_4_txn_amt_pct', 'card_5_txn_amt_pct', 'card_6_txn_amt_pct', 'card_7_txn_amt_pct', 'card_8_txn_amt_pct',
            'card_9_txn_amt_pct', 'card_10_txn_amt_pct', 'card_11_txn_amt_pct', 'card_12_txn_amt_pct', 'card_13_txn_amt_pct',
            'card_14_txn_amt_pct', 'card_other_txn_amt_pct']

df = pd.read_csv('tbrain_cc_training_48tags_hash_final.csv', usecols=use_cols)

df = df[df.shop_tag.isin(pred_class)]
df['max_card_idx'] = np.argmax(df.values[:,9:], axis = 1)+1

all_chid_list = [i for i in range(10000000, 10000000+500000)]

data = pd.DataFrame(np.zeros((500000, 24 * len(pred_class) * 7)))
data.columns = [f"txn_{k}_dt{i}_shoptag_{j}" for i in range(1, 25) for j in pred_class for k in ['amt', 'cnt', 'card', 'off_do', 'on_do', 'off_ov', 'on_ov']]
data["chid"] = all_chid_list

data.index = data.chid
data.drop("chid", inplace=True, axis=1)

for (date, shop_tag), date_shoptag_data in df.groupby(["dt", "shop_tag"]):
    data.loc[date_shoptag_data.chid.values, [f"txn_amt_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.txn_amt.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_cnt_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.txn_cnt.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_card_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.max_card_idx.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_off_do_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.domestic_offline_amt_pct.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_on_do_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.domestic_online_amt_pct.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_off_ov_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.overseas_offline_amt_pct.values.reshape(-1,1)
    data.loc[date_shoptag_data.chid.values, [f"txn_on_ov_dt{date}_shoptag_"+shop_tag]] = date_shoptag_data.overseas_online_amt_pct.values.reshape(-1,1)

data.to_csv('cnt_amt_card_onoff_doov_data.csv')